**0. Instalar las librerias necesarias**

In [36]:
!pip install PyPDF2
#!pip install gensim
!pip install pdfminer.six

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**1. Importar las librerias necesarias**

In [37]:
import nltk
from  os import path,listdir,mkdir,rename,sep #estos modulos les permiten obtener el listado de los archivos de una carpeta, crear una carpeta y renombrar archivos
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
#from gensim import keywords

#import PyPDF2
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text

from collections import Counter
import re

import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt

import Crypto

In [38]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/juancho/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [39]:
#drive.mount('/content/drive/')
nltk.download('stopwords')
nltk.download('punkt')
stopws = set(stopwords.words('spanish'))

#files = listdir(direccion)
#print(files)

#Para quitar el "Copia de "
#for name in archivos:
#  rename(direccion+name,direccion+name[9:])


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/juancho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/juancho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**2. Definir funciones**

In [40]:
def converter_PyPDF2(namefiledir):
  file = open(namefiledir, "rb")
  pdf = PyPDF2.PdfFileReader(file)

  num_pages = pdf.numPages

  count = 0

  text = ""

  while count < num_pages:
    pageObj = pdf.getPage(count)
    count +=1
    try:
      text += pageObj.extractText()
    except:
      text = text

  return(text)

def limpiartexto(texto):
  referencesOUT = True
  refined = True#False
  longitud_minima_esperado_de_lines = 35
  numero_de_stopword_esperados_de_lines = 1
  numero_de_stopword_esperados_de_3_lines = 2

  texto_limpio = ""

  listarenglones = texto.split("\n")

  for indice_reglon in range(len(listarenglones)-1):
    if len(listarenglones[indice_reglon]) and listarenglones[indice_reglon][-1]=="-":
      #print("#"+listarenglones[indice_reglon]+"#", listarenglones[indice_reglon][-1])
      #print("#continua#"+listarenglones[indice_reglon+1]+"#")
      continuacion_palabra = listarenglones[indice_reglon+1][0:listarenglones[indice_reglon+1].find(" ")]

      if listarenglones[indice_reglon][-2:]==" -":
        listarenglones[indice_reglon] = listarenglones[indice_reglon][:-2]+continuacion_palabra
        listarenglones[indice_reglon+1] = listarenglones[indice_reglon+1][len(continuacion_palabra):]
      if listarenglones[indice_reglon][-1]=="-":
        listarenglones[indice_reglon] = listarenglones[indice_reglon][:-1]+continuacion_palabra
        listarenglones[indice_reglon+1] = listarenglones[indice_reglon+1][len(continuacion_palabra):]
      #print("#2"+listarenglones[indice_reglon]+"#2")

  texto_limpio = "\n".join(listarenglones)
  text = texto_limpio


  text_refined = str()

  if referencesOUT:
    startreferences = -1

    if startreferences < 0: startreferences = text.rfind("REFERENCES")-len("REFERENCES")
    if startreferences < 0: startreferences = text.rfind("References")-len("References")
    if startreferences < 0: startreferences = text.rfind("R E F E R E N C E S")-len("R E F E R E N C E S")
    if startreferences < 0: startreferences = text.rfind("R e f e r e n c e s")-len("R e f e r e n c e s")
    if startreferences < 0: startreferences = text.rfind("Literature Cited")-len("Literature Cited")

    #if startreferences < 0: print("How no!!")

    if startreferences > 0: text = text[:startreferences]


  if not refined:
    text_refined = text
  else:
    lines = text.split("\n")
    for nline in range(len(lines)):

      line = lines[nline]
      validline = 0

      preline = lines[nline-1:nline]
      if len(preline)>0 : preline = preline[0]
      else: preline = ""
      postline = lines[nline+1:nline+2]
      if len(postline)>0 : postline = postline[0]
      else: postline = ""


      if len(line) > longitud_minima_esperado_de_lines:
        validline += 1
      elif len(preline) > longitud_minima_esperado_de_lines: #and line.find(".")>0:
        validline += 1

      #print("#", centroid_plusExterior_listline)
      if len(set(line.split(" ")) & stopws)  > numero_de_stopword_esperados_de_lines:
        validline += 1
      else:
        triline = preline + " " + line + " " + postline
        if len(set(triline.split(" ")) & stopws)  > numero_de_stopword_esperados_de_3_lines:
          validline += 1

      if validline > 1:
        text_refined += line+" " #+"\n"

    texto=text_refined

    texto = texto.lower()

    texto = word_tokenize(texto)

    texto = list(filter(lambda a: not a in (stopws and set(["et","al"])), texto))
    texto = list(filter(lambda a: a.isalpha(), texto))
    texto = " ".join(texto)

  return(texto)

In [41]:
def read_pypdf2(namefile):
  reader = PdfReader(namefile)
  full_text = ""

  if reader.is_encrypted:
    try:
      reader.decrypt('')
    except:
      print("Could not decrypt the PDF file.")
      return(full_text)

  for page in reader.pages:
    full_text += page.extract_text()

  return(full_text)

In [42]:
def extract_cont_word(text):
  voc_count=Counter(text.split())
  return(voc_count)


**3. Variables glabales o de entrada**


In [43]:
direccion="./"
#direccion_pdf="./all_pages/"
direccion_pdf = "/home/juancho/aerrors/"
direccion_txt="./all_pages_clean/txt/"
direccion_txtpypdf="./all_pages_clean/pypdf/"
lenguaje='spanish'

In [44]:
files = listdir(direccion_pdf)
print(files)

['0484.html', '0041.html', '0425.html', '0487.html', '0087.pdf', '0560.pdf', '0249.pdf', '0308.pdf', '0375.html', '0313.html', '0069.html', '0292.html', '0473.pdf', '0485.html', '0402.html', '0044.mp4', '0500.pdf', '0498.pdf', '0228.html', '0336.html', '0315.pdf', '0254.pdf', '0505.html', '0475.pdf', '0172.html', '0175.pdf', '0342.mp4', '0345.mp4', '0225.html', '0508.pdf']


**4. Abrir las carpetas con los archivos**

In [45]:
print(stopws)

{'sobre', 'tuya', 'estada', 'haya', 'sea', 'fuéramos', 'fuese', 'estuvisteis', 'estando', 'estábamos', 'soy', 'no', 'tú', 'tendría', 'hubieran', 'yo', 'ese', 'hubo', 'mucho', 'fuera', 'e', 'suyo', 'habíais', 'estaremos', 'habíamos', 'eran', 'ni', 'hubimos', 'eso', 'muy', 'estaban', 'algunos', 'estad', 'una', 'son', 'fuerais', 'hubiéramos', 'esos', 'somos', 'fue', 'hubieses', 'todo', 'mías', 'fuimos', 'sentid', 'sintiendo', 'estuve', 'tenga', 'tendré', 'habidos', 'habrás', 'habiendo', 'tengan', 'sería', 'para', 'seríamos', 'será', 'les', 'esa', 'hubiésemos', 'suya', 'tuviese', 'te', 'habido', 'del', 'tanto', 'habían', 'tuvieseis', 'estáis', 'fueran', 'habré', 'estamos', 'estarás', 'fuésemos', 'su', 'estuviésemos', 'tuyos', 'había', 'sean', 'qué', 'hubierais', 'y', 'habida', 'le', 'estarías', 'tenidas', 'tendríais', 'nuestras', 'estuviera', 'míos', 'estadas', 'estaríamos', 'hubisteis', 'hubieseis', 'tuvieras', 'tenida', 'esas', 'uno', 'habríais', 'estén', 'un', 'habrán', 'los', 'teníais'

**5. Extraer el texto del pdf**

In [46]:
files = listdir(direccion_pdf)
if not path.exists(direccion_txt):
  mkdir(direccion_txt)
print(len(files))
#print(files)
for file in files:
  #print(direccion_pdf+file)
  if path.isfile(direccion_pdf+file) and file[-4:] == ".pdf":
    #print(direccion_pdf+file)
    print("Processing file:", file)
    text = extract_text(direccion_pdf+file)
    file_txt = open(direccion_txt+file[:-4]+".txt", "w")
    file_txt.write(text)
    file_txt.close()
    print(file)

30
Processing file: 0087.pdf
0087.pdf
Processing file: 0560.pdf
0560.pdf
Processing file: 0249.pdf
0249.pdf
Processing file: 0308.pdf
0308.pdf
Processing file: 0473.pdf
0473.pdf
Processing file: 0500.pdf
0500.pdf
Processing file: 0498.pdf
0498.pdf
Processing file: 0315.pdf
0315.pdf
Processing file: 0254.pdf
0254.pdf
Processing file: 0475.pdf


Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value


0475.pdf
Processing file: 0175.pdf


Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value


0175.pdf
Processing file: 0508.pdf
0508.pdf


In [47]:
files = listdir(direccion_pdf)
print(len(files))
print(files)
if not path.exists(direccion_txtpypdf):
  mkdir(direccion_txtpypdf)

for file in files:
 if path.isfile(direccion_pdf+file) and file[-4:] == ".pdf":
    print(file)
    try:
      text = read_pypdf2(direccion_pdf+file)
      file_txt = open(direccion_txtpypdf+file[:-4]+".txt", "w", encoding='utf-8', errors='ignore')
      file_txt.write(text)
      file_txt.close()
    except Exception as e:
      print("Error processing file:", file)
      print(e)


30
['0484.html', '0041.html', '0425.html', '0487.html', '0087.pdf', '0560.pdf', '0249.pdf', '0308.pdf', '0375.html', '0313.html', '0069.html', '0292.html', '0473.pdf', '0485.html', '0402.html', '0044.mp4', '0500.pdf', '0498.pdf', '0228.html', '0336.html', '0315.pdf', '0254.pdf', '0505.html', '0475.pdf', '0172.html', '0175.pdf', '0342.mp4', '0345.mp4', '0225.html', '0508.pdf']
0087.pdf
0560.pdf
0249.pdf
0308.pdf
0473.pdf
0500.pdf
0498.pdf
0315.pdf
0254.pdf
0475.pdf
0175.pdf
0508.pdf


**6.Seleccionar txt de extracción**

In [51]:
def count_long_words(counter, max_length=10):
    """
    Cuenta cuántas palabras en un Counter tienen más de `min_length` letras.

    Args:
        counter (Counter): frecuencias de palabras.
        min_length (int): longitud mínima de palabra (default=10).

    Returns:
        int: número de palabras que cumplen la condición.
        list: lista de esas palabras (opcional para revisión).
    """
    excluded_patterns = ("http", "www", ".com", "@", "doi")


    #long_words = [word for word in counter if len(word) > max_length]
    #long_words = [word for word in counter if len(word) > max_length and not any(pat in word.lower() for pat in excluded_patterns)]
    long_words = []

    for word in counter:
      if len(word) > max_length and not any(pat in word.lower() for pat in excluded_patterns):
        # contar cuántos caracteres son letras
        letters = sum(1 for ch in word if ch.isalpha())
        ratio = letters / len(word)

        if ratio >= 0.5:  # al menos 50% letras
            long_words.append(word)

    return len(long_words), long_words

In [52]:
files = listdir(direccion_txt)
print(len(files))
print(files)


for file in files:
 if path.isfile(direccion_txt+file) and file[-4:] == ".txt":
    file_txt = open(direccion_txt+file, "r")
    text = file_txt.read()
    voc=extract_cont_word(text)
    sizedoc=sum(voc.values())
    file_txt.close()
    #print(sizedoc)
    frecuenciapromedio=sizedoc/(len(voc) + 1)
    #print(frecuenciapromedio)
    bajoumbral = {word: count for word, count in voc.items() if count > frecuenciapromedio}
    #print("Palabras más frencuentes que el promedio:", len(bajoumbral)/len(voc))
    razonpalabrasfrecuentes=len(bajoumbral)/(len(voc) + 1)

    if 0.1 > razonpalabrasfrecuentes or razonpalabrasfrecuentes > 0.2:
      print("valor anomalo",file)
      print(razonpalabrasfrecuentes)
      print(voc)

    #print(valida_zipf(voc, top_n=400, plot=False))
    max_length=20
    if count_long_words(voc, max_length)[0] > 20:
      print("\n\nNumero de palabras largas:", count_long_words(voc, max_length))
      print(voc,"\n\n\n\n")
    max=0
    for w in voc:
      if len(w) > max:
        max=len(w)

    print("nuevas",len(voc),max)


12
['0249.txt', '0498.txt', '0473.txt', '0087.txt', '0508.txt', '0475.txt', '0254.txt', '0315.txt', '0308.txt', '0500.txt', '0560.txt', '0175.txt']
valor anomalo 0249.txt
0.0
Counter()
nuevas 0 0
nuevas 2194 21
valor anomalo 0473.txt
0.0
Counter()
nuevas 0 0
nuevas 4737 20
nuevas 2681 24
nuevas 3700 72
nuevas 1987 42
nuevas 1812 21
nuevas 2101 74
nuevas 1703 65
nuevas 3035 34
nuevas 2435 409


In [56]:
files = listdir(direccion_txt)

for file in files[:10]:
 if path.isfile(direccion_txt+file) and path.isfile(direccion_txtpypdf+file) and file[-4:] == ".txt":
    file_txt = open(direccion_txt+file, "r")
    file_txtpypdf = open(direccion_txtpypdf+file, "r")
    text = file_txt.read()
    textpypdf = file_txtpypdf.read()
    voc=extract_cont_word(text)
    vocpypdf=extract_cont_word(textpypdf)
    sizedoc=sum(voc.values())
    sizedocpypdf=sum(vocpypdf.values())
    file_txt.close()
    file_txtpypdf.close()
    #print(sizedoc)
    frecuenciapromedio=sizedoc/(len(voc) + 1)
    frecuenciapromediopypdf=sizedocpypdf/(len(vocpypdf) + 1)
    #print(frecuenciapromedio)
    bajoumbral = {word: count for word, count in voc.items() if count > frecuenciapromedio}
    bajoumbralpypdf = {word: count for word, count in vocpypdf.items() if count > frecuenciapromediopypdf}
    #print("Palabras más frencuentes que el promedio:", len(bajoumbral)/len(voc))
    razonpalabrasfrecuentes=len(bajoumbral)/(len(voc) + 1)
    razonpalabrasfrecuentespypdf=len(bajoumbralpypdf)/(len(vocpypdf) + 1)


    #if 0.1 > razonpalabrasfrecuentes or razonpalabrasfrecuentes > 0.2:
    #  print("valor anomalo",file)
    #  print(razonpalabrasfrecuentes)
    #  print(voc)

    #print(valida_zipf(voc, top_n=400, plot=False))
    max_length=20
    #if count_long_words(voc, max_length)[0] > 20:
    #  print("\n\nNumero de palabras largas:", count_long_words(voc, max_length))
    #  print(voc,"\n\n\n\n")

    num_long_words = count_long_words(voc, max_length)[0]
    num_long_wordspypdf = count_long_words(vocpypdf, max_length)[0]

    #if num_long_words > 20 or num_long_wordspypdf > 20 or 0.1 > razonpalabrasfrecuentes or razonpalabrasfrecuentes > 0.2 or 0.1 > razonpalabrasfrecuentespypdf or razonpalabrasfrecuentespypdf > 0.2:
    if False:
      print("\n\nNumero de palabras largas:", num_long_words, num_long_wordspypdf)
      print("valor anomalo",file)
      print(razonpalabrasfrecuentes, razonpalabrasfrecuentespypdf)
      print(voc)
      print(vocpypdf)
      print("Zipf",valida_zipf(voc, top_n=400, plot=False),valida_zipf(vocpypdf, top_n=400, plot=Fakse))




In [57]:
def valida_zipf(counter, top_n=50, plot=True):
    """
    Valida ley de Zipf

    """
    # palabras
    freqs = np.array([c for _, c in counter.most_common(top_n)])
    ranks = np.arange(1, len(freqs) + 1)

    # Log transform
    log_ranks = np.log(ranks)
    log_freqs = np.log(freqs)

    # Correlation (should be close to -1 if Zipf’s law holds)
    correlation = np.corrcoef(log_ranks, log_freqs)[0,1]

    if plot:
        plt.scatter(log_ranks, log_freqs)
        plt.plot(log_ranks, np.poly1d(np.polyfit(log_ranks, log_freqs, 1))(log_ranks), color="red")
        plt.xlabel("log(Rank)")
        plt.ylabel("log(Frequency)")
        plt.title(f"Zipf's Law Validation (corr={correlation:.2f})")
        plt.show()

    return {"correlation": correlation, "top_n": top_n}

In [58]:
files = listdir(direccion_txt)

direccion_txt_best="./all_pages_clean/best/"

if not path.exists(direccion_txt_best):
  mkdir(direccion_txt_best)

for file in files:
 if path.isfile(direccion_txt+file) and path.isfile(direccion_txtpypdf+file) and file[-4:] == ".txt":
    file_txt = open(direccion_txt+file, "r")
    file_txtpypdf = open(direccion_txtpypdf+file, "r")
    text = file_txt.read()
    textpypdf = file_txtpypdf.read()
    voc=extract_cont_word(text)
    vocpypdf=extract_cont_word(textpypdf)
    print(file,"Zipf",valida_zipf(voc, top_n=400, plot=False),valida_zipf(vocpypdf, top_n=400, plot=False))
    Zipf=valida_zipf(voc, top_n=400, plot=False)
    Zipfpypdf=valida_zipf(vocpypdf, top_n=400, plot=False)
    if Zipf["correlation"] < Zipfpypdf["correlation"]:
      file_txt_best = open(direccion_txt_best+file, "w")
      file_txt_best.write(text)
      file_txt_best.close()
    else:
      file_txt_best = open(direccion_txt_best+file, "w")
      file_txt_best.write(textpypdf)
      file_txt_best.close()

print(len(listdir(direccion_txt_best)))

0249.txt Zipf {'correlation': nan, 'top_n': 400} {'correlation': nan, 'top_n': 400}
0498.txt Zipf {'correlation': -0.9940212145598855, 'top_n': 400} {'correlation': -0.9946448844729764, 'top_n': 400}
0473.txt Zipf {'correlation': nan, 'top_n': 400} {'correlation': nan, 'top_n': 400}
0087.txt Zipf {'correlation': -0.9955977091060156, 'top_n': 400} {'correlation': -0.995305403335225, 'top_n': 400}
0508.txt Zipf {'correlation': -0.9917101343839825, 'top_n': 400} {'correlation': -0.9911748510549057, 'top_n': 400}
0475.txt Zipf {'correlation': -0.9893456358191915, 'top_n': 400} {'correlation': -0.9893692045738376, 'top_n': 400}
0254.txt Zipf {'correlation': -0.9863998298964739, 'top_n': 400} {'correlation': -0.9873569614576224, 'top_n': 400}
0315.txt Zipf {'correlation': -0.9902824240071233, 'top_n': 400} {'correlation': -0.9898034723023286, 'top_n': 400}
0308.txt Zipf {'correlation': -0.9906826244754292, 'top_n': 400} {'correlation': -0.9909841423257528, 'top_n': 400}
0500.txt Zipf {'corre

/home/juancho/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/juancho/.local/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/juancho/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/juancho/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/juancho/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


**7. Limpieza y preprocesamiento de textos**

In [59]:
direccion_txt_best="./all_pages_clean/best/"

files_txt_best = listdir(direccion_txt_best)
documentos=[]
name_documentos=[]
documentos_all=[]
for file in files_txt_best:
 if path.isfile(direccion_txt_best+file) and file[-4:] == ".txt":
    print("####",file)
    file_content = open(direccion_txt_best+file, "r")
    texto = file_content.read()
    #voc=extract_cont_word(texto)
    #print(voc)
    file_content.close()
    #print("&1",texto[:500])
    #print("$1",len(texto))
    texto = limpiartexto(texto)
    #print(texto)
    name_documentos.append(file)
    documentos.append(texto)
    documentos_all.append([file,texto])
    #print("$2",len(texto))
    #print("&2",texto[:500])

#print(documentos_all)

#### 0249.txt
#### 0498.txt
#### 0473.txt
#### 0087.txt
#### 0508.txt
#### 0475.txt
#### 0254.txt
#### 0315.txt
#### 0308.txt
#### 0500.txt
#### 0560.txt
#### 0175.txt
